# Testing of Rs. 2000 currency notes

In [ ]:
# Importing all necessary libraries

import cv2                            # Importing Opencv library for image processing and computer vision
import numpy as np                    # Importing numpy library 
import matplotlib.pyplot as plt       # Importing matplotlib library to plot the images directly in notebook
import os
import tkinter as tk
from skimage.metrics import structural_similarity as ssim   # Importing ssim calculating modules from skimage library


# Importing tkinter library to build GUI
from tkinter import *
from tkinter.ttk import Progressbar
from tkinter import font

import time

#Resizing the Plots
plt.rcParams["figure.figsize"] = (12, 12)

In [ ]:
# Declaring variable for progress bar to store the total progress
myProgress =0.0

In [ ]:
# This Ipython magic will retrieve the path of the input image which is stored when the gui_1.ipynb is executed.

%store -r path

# A sample path:
# path = r'Dataset\2000_dataset\2000_s3.jpg'

print('Path of input image: ', path)

In [ ]:
# Reading the image
test_img = cv2.imread(path)

# Pre- processing

In [ ]:
# Pre- processing

# Resizing the image
test_img = cv2.resize(test_img, (1165,455))

# Guassian Blur
blur_test_img = cv2.GaussianBlur(test_img, (5,5), 0)

# Grayscale conversion
gray_test_image = cv2.cvtColor(blur_test_img, cv2.COLOR_BGR2GRAY)

def preprocessing():
    # Showing original currency note
    plt.imshow(gray_test_image, 'gray')
    plt.title('Input image after pre- processing')
    plt.show()
    
    # Updating the progress
    progress['value']=5
    ProgressWin.update_idletasks()

In [ ]:
#  Calculating SSIM of the two images sent as parameters

def calculateSSIM(template_img, query_img):
    min_w = min(template_img.shape[1], query_img.shape[1])
    min_h = min(template_img.shape[0], query_img.shape[0])
    
    # Resizing the two images so that both have same dimensions
    img1 = cv2.resize(template_img, (min_w, min_h))
    img2 = cv2.resize(query_img, (min_w, min_h))
    
    # Conversion to gray- scale
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Plotting the images
    plt.subplot(1, 2, 1)
    plt.imshow(img1, 'gray')

    plt.subplot(1, 2, 2)
    plt.imshow(img2, 'gray')

    plt.show()
    
    # Find the SSIM score and return
    score = ssim(img1, img2)
    return (1-score)

In [ ]:
# Feature detection using ORB

def computeORB(template_img, query_img):
    # ===================== Creating orb object ==============================

    nfeatures=700;
    scaleFactor=1.2;
    nlevels=8;
    edgeThreshold=15; # Changed default (31);

    # Initialize the ORB detector algorithm 
    orb = cv2.ORB_create(
        nfeatures,
        scaleFactor,
        nlevels,
        edgeThreshold)
    # =========================================================================
    
    # Find the keypoints and descriptors with ORB
    # This will find the keypoints of each of the image and then find the descriptors corresponding to each keypoint.
    
    kpts1, descs1 = orb.detectAndCompute(template_img,None)
    kpts2, descs2 = orb.detectAndCompute(query_img,None)
    
    # ====================================== Brute Force Matching ==============================
    # Starting a brute force matcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Finding matches between the 2 descrptor sets
    matches = bf.match(descs1, descs2)

    # sort the matches in the order of their distance
    # Lower the distance, better the matching
    dmatches = sorted(matches, key = lambda x:x.distance)

    # ===========================================================================================
    
    # ======================================= Image homography ================================
    ## extract the matched keypoints
    src_pts  = np.float32([kpts1[m.queryIdx].pt for m in dmatches]).reshape(-1,1,2)
    dst_pts  = np.float32([kpts2[m.trainIdx].pt for m in dmatches]).reshape(-1,1,2)
        
    ## find homography matrix and do perspective transform
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    h,w = template_img.shape[:2]
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    
    if M is not None:
        dst = cv2.perspectiveTransform(pts,M)
    else:
        dst = None

    # This finds the template region in the test currency note
    # ====================================================================================
    
    # Returning necessary data
    return dst, dst_pts, kpts1, kpts2, dmatches

# Features 1- 7: Using ORB and SSIM

In [ ]:
# Initialize global variables
score_set_list = []                # Stores the SSIM score set of each feature
best_extracted_img_list = []       # Stores the extracted image with the highest SSIM score for each feature
avg_ssim_list = []                 # Stores the avg SSIM value for each feature
search_area_list = [
    [10, 100, 10, 100],  # Search area for feature 1
    [20, 110, 20, 110],  # Search area for feature 2
    [30, 120, 30, 120],  # Search area for feature 3
    [40, 130, 40, 130],  # Search area for feature 4
    [50, 140, 50, 140],  # Search area for feature 5
    [60, 150, 60, 150],  # Search area for feature 6
    [70, 160, 70, 160]   # Search area for feature 7
]
feature_area_limits_list = [
    (0, 20000),  # Limits for Feature 1
    (0, 20000),  # Limits for Feature 2
    (0, 20000),  # Limits for Feature 3
    (0, 20000),  # Limits for Feature 4
    (0, 20000),  # Limits for Feature 5
    (0, 20000),  # Limits for Feature 6
    (0, 20000)   # Limits for Feature 7
]

def testFeature_1_2_7():
    NUMBER_OF_TEMPLATES = 6
    NUM_OF_FEATURES = 7
    global score_set_list
    global best_extracted_img_list
    global avg_ssim_list

    # Progress bar
    myProgress = progress.cget('value')  # Get the current progress value

    # Iterating for each feature
    for j in range(NUM_OF_FEATURES):
        print('ANALYSIS OF FEATURE ' + str(j + 1))

        score_set = []           # SSIM scores for each template of current feature will be stored here
        max_score = -1           # Stores max SSIM score
        max_score_img = None     # Stores extracted image with max SSIM score for the current feature

        # Performing feature detection, extraction, and comparison for each template
        for i in range(NUMBER_OF_TEMPLATES):
            print('---> Template ' + str(i + 1) + ' :')

            # **Modified Path: Use Absolute Path**
            template_path = fr'C:\Users\PETETI HARSHITHA\Downloads\Dataset\Data Set\Dataset\2000_features dataset\Feature {j + 1}\{i + 1} ({j}).jpg'
            template_img = cv2.imread(template_path)

            # **Added Error Handling: Check if Image Loaded Successfully**
            if template_img is None:
                print(f"Error: Could not load template image {template_path}")
                continue

            template_img_blur = cv2.GaussianBlur(template_img, (5, 5), 0)
            template_img_gray = cv2.cvtColor(template_img_blur, cv2.COLOR_BGR2GRAY)
            test_img_mask = gray_test_image.copy()

            # Creating a mask to search the current template.
            search_area = search_area_list[j]
            test_img_mask[:, :search_area[0]] = 0
            test_img_mask[:, search_area[1]:] = 0
            test_img_mask[:search_area[2], :] = 0
            test_img_mask[search_area[3]:, :] = 0

            # Feature detection using ORB 
            dst, dst_pts, kpts1, kpts2, dmatches = computeORB(template_img_gray, test_img_mask)

            # Error handling for Homography matrix
            if dst is None:
                print('An Error occurred - Homography matrix is of NoneType')
                continue

            query_img = test_img.copy()

            # Drawing polygon around the detected region on the test currency note
            res_img1 = cv2.polylines(query_img, [np.int32(dst)], True, (0, 0, 255), 1, cv2.LINE_AA)

            # Drawing match lines between the matched descriptors
            res_img2 = cv2.drawMatches(template_img, kpts1, res_img1, kpts2, dmatches[:20], None, flags=2)

            # Find the details of a bounding rectangle that bounds the above polygon --- green rectangle
            (x, y, w, h) = cv2.boundingRect(dst)  # Details about the rectangle that bounds this contour  

            # Debugging output
            print(f"Bounding Box: x={x}, y={y}, w={w}, h={h}, area={w*h}")

            # Checking if the area of the detected region is within the min and max area allowed for current feature 
            min_area, max_area = feature_area_limits_list[j]
            feature_area = w * h

            if feature_area < min_area or feature_area > max_area:
                # Discard current template without attempting a second bounding rectangle
                print(f'Template Discarded - Area {feature_area} is outside permitted range!')
                continue

            # Draw the rectangle
            cv2.rectangle(res_img1, (x, y), (x + w, y + h), (0, 255, 0), 3)

            # Plotting images
            plt.rcParams["figure.figsize"] = (16, 16)
            plt.subplot(1, 2, 1)
            plt.imshow(cv2.cvtColor(res_img2, cv2.COLOR_BGR2RGB))
            plt.subplot(1, 2, 2)
            plt.imshow(cv2.cvtColor(res_img1, cv2.COLOR_BGR2RGB))
            plt.show()

            # SSIM calculation
            # Ensure the bounding rectangle is within image boundaries
            if y < 0 or x < 0 or (y + h) > blur_test_img.shape[0] or (x + w) > blur_test_img.shape[1]:
                print(f"Invalid crop dimensions: x={x}, y={y}, w={w}, h={h}")
                continue  # Skip this iteration if the dimensions are invalid

            crop_img = blur_test_img[y:y + h, x:x + w]

            # Check if crop_img is empty
            if crop_img.size == 0:
                print("Warning: crop_img is empty, skipping resizing and SSIM calculation.")
                continue  # Skip this iteration if the cropped image is empty

            # Log dimensions for debugging
            print(f"Template dimensions: {template_img_blur.shape}")
            print(f"Crop dimensions: {crop_img.shape}")

            # Ensure crop_img has the same dimensions as template_img_blur
            if crop_img.shape != template_img_blur.shape:
                # Resize crop_img to match template_img_blur
                try:
                    crop_img = cv2.resize(crop_img, (template_img_blur.shape[1], template_img_blur.shape[0]))
                    print(f"Resized crop_img to match template_img_blur dimensions: {crop_img.shape}")
                except Exception as e:
                    print(f"Error resizing crop_img: {e}")
                    continue  # Skip this iteration if resizing fails

            # SSIM calculation
            score = calculateSSIM(template_img_blur, crop_img)
            score_set.append(score)
            print('SSIM score: ', score, '\n')

            # Keeping details about extracted region with highest SSIM score
            if score > max_score:
                max_score = score
                max_score_img = crop_img

            # Progress bar - Updating the progress
            myProgress += (75.0 / (NUM_OF_FEATURES * NUMBER_OF_TEMPLATES))
            progress['value'] = myProgress 
            ProgressWin.update_idletasks()

        # Storing necessary data
        score_set_list.append(score_set)
        print('SSIM score set of Feature ' + str(j + 1) + ': ', score_set, '\n')

        if len(score_set) != 0:
            avg_ssim = sum(score_set) / len(score_set)
            avg_ssim_list.append(avg_ssim)
            print('Average SSIM of Feature ' + str(j + 1) + ': ', avg_ssim, '\n')
        else:
            print('No SSIM scores were found for this feature!')
            avg_ssim_list.append(0.0)
            print('Average SSIM of Feature ' + str(j + 1) + ': 0', '\n')

        best_extracted_img_list.append([max_score_img, max_score])

    # Printing all details for features 1-7
    print('Final Score-set list:\n')

    for x in range(len(score_set_list)):
        print('Feature', x + 1, ':', score_set_list[x])
    print('\n')

    print('Final Average SSIM list for each feature:\n')

    for x in range(len(avg_ssim_list)):
        print('Feature', x + 1, ':', avg_ssim_list[x])


In [ ]:
left_BL_result = []
right_BL_result = []
result_list = []
number_panel_result = []


# Feature 8: Left Bleed Lines

In [ ]:
# Function to count number of bleed lines in left side- Feature 8
# Algo 2

def testFeature_8():
    plt.rcParams["figure.figsize"] = (5, 5)

    # Check Feature 8- Left bleed lines
    print('\nANALYSIS OF FEATURE 8 : LEFT BLEED LINES\n')
    
    # Cropping the region in which left bleed lines are present- Feature extraction
    crop = test_img[80:230, 10:30]

    img = crop.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Thresholding the image
    _, thresh = cv2.threshold(gray, 130, 255, cv2.THRESH_BINARY)

    plt.imshow(thresh, 'gray')
    plt.show()
    
    whitePixelValue = 255      # White pixel   
    blackPixelValue = 0        # Black pixel

    width = thresh.shape[1]    # width of thresholded image

    # Result will be stored here
    result = []                # will contain the number of black regions in each column (if the colums is non- erroneos)
    num_of_cols = 0            # will contain the number of non- erroneos columns

    # Non erroneous columns are those columns which contains less noise.

    print('Number of black regions found in each column: ')
    
    # iteration over each column in the cropped image
    for j in range(width):
        col =thresh[:, j:j+1]     # Extracting each column of thresholded image
        count = 0                 # Counter to count number of black regions in each extracted column
        
        # Iterating over each row (or pixel) in the current columm
        for i in range(len(col)-1):

            # Taking two consecutive pixels and storing their intensity value
            pixel1_value = col[i][0]
            pixel2_value = col[i+1][0]

            #----------------------------------------------
            # This part modifies any error pixels, if present.
            # Actually in a binary threshold, all pixels should be either white or black.
            # If due to some error pixels other than white or black are present, then the pixel is taken as white pixel

            if pixel1_value != 0 and pixel1_value != 255:
                pixel1_value = 255
            if pixel2_value != 0 and pixel2_value != 255:
                pixel2_value = 255

            #-------------------------------------------------


            # If current pixel is white and next pixel is black, then increment the counter.
            # This shows that a new black region has been discovered.
            if pixel1_value == whitePixelValue and pixel2_value == blackPixelValue:
                count += 1

        # If the counter is less than 10, it is a valid column. (less noise is present)
        if count > 0 and count < 10:
            print(count)
            result.append(count)
            num_of_cols += 1
        else:
            # discard the count if it is too great e.g. count> 10 (Erroneous Column)
            # This removes/ drops those columns which contain too much noise
            print(count, 'Erroneous -> discarded') 
    
    # Printing necessary details
    print('\nNumber of columns examined: ', width)
    print('Number of non- erroneous columns found: ', num_of_cols)
    
    if num_of_cols != 0:
        average_count = sum(result)/num_of_cols
    else:
        average_count = -1
        print('Error occured- Division by 0')

    print('\nAverage number of black regions is: ', average_count)
    
    # Storing the thresholded image and average number of bleed lines detected 
    global left_BL_result
    left_BL_result = [thresh, average_count]
    
    # Updating progess in progress bar
    global myProgress
    progress['value']=80
    ProgressWin.update_idletasks()

# Feature 9: Right Bleed Lines

In [ ]:
def testFeature_9():
    plt.rcParams["figure.figsize"] = (5, 5)

    print('\nANALYSIS OF FEATURE 9 : RIGHT BLEED LINES\n')
    
    # Cropping the region in which left bleed lines are present- Feature extraction
    crop = test_img[90:230, 1140:1160]

    img = crop.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Thresholding the image
    _, thresh = cv2.threshold(gray, 130, 255, cv2.THRESH_BINARY)

    plt.imshow(thresh, 'gray')
    plt.show()

    whitePixelValue = 255      # White pixel   
    blackPixelValue = 0        # Black pixel

    width = thresh.shape[1]    # width of thresholded image

    result = []                # will contain the number of black regions in each column (if the columns are non-erroneous)
    num_of_cols = 0            # will contain the number of non-erroneous columns

    print('Number of black regions found in each column: ')
    
    for j in range(width):
        col = thresh[:, j:j+1]     # Extracting each column of thresholded image
        count = 0                  # Counter to count number of black regions in each extracted column
        
        for i in range(len(col)-1):
            pixel1_value = col[i][0]
            pixel2_value = col[i+1][0]

            # Modifying erroneous pixel values
            pixel1_value = 255 if pixel1_value not in (0, 255) else pixel1_value
            pixel2_value = 255 if pixel2_value not in (0, 255) else pixel2_value

            # Count black regions
            if pixel1_value == whitePixelValue and pixel2_value == blackPixelValue:
                count += 1

        if 0 < count < 10:  # Valid column check
            print(count)
            result.append(count)
            num_of_cols += 1
        else:
            print(count, 'Erroneous -> discarded')

    print('\nNumber of columns examined: ', width)
    print('Number of non-erroneous columns found: ', num_of_cols)

    average_count = sum(result) / num_of_cols if num_of_cols != 0 else -1
    print('\nAverage number of black regions is: ', average_count)

    global right_BL_result
    right_BL_result = [thresh, average_count]
    
    global myProgress
    progress['value'] = 85
    ProgressWin.update_idletasks()


# Feature 10: Currency Number Panel

In [ ]:
plt.rcParams["figure.figsize"] = (7, 7)

# Cropping out the number panel
crop = gray_test_image[360:440, 760:1080]    # Cropped gray image
crop_bgr = test_img[360:440, 760:1080]       # Cropped BGR image


In [ ]:
def testFeature_10():
    plt.imshow(crop_bgr)
    plt.show()

    plt.rcParams["figure.figsize"] = (7, 7)

    print('\n\nANALYSIS OF FEATURE 10 : NUMBER PANEL \n')

    test_passed = False
    res_img_list = []
    count = 0
    num = 1
    
    # Loop through threshold values
    for thresh_value in range(90, 155, 5):
        _, thresh = cv2.threshold(crop, thresh_value, 255, cv2.THRESH_BINARY)

        print('---> Threshold ' + str(num) + ' with Threshold value ' + str(thresh_value) + ' :')
        num += 1

        copy = crop_bgr.copy()
        img = cv2.bitwise_and(crop, crop, mask=thresh)
        contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        h_img, w_img = img.shape[:2]
        bounding_rect_list = []    

        for contour in contours:
            [x, y, w, h] = cv2.boundingRect(contour)
            
            # Calculate area and check if it's within allowed range
            area = w * h
            min_area = 0     # Minimum area for a valid rectangle
            max_area = 10000    # Maximum area (optional, adjust as needed)

            if x != 0:
                bounding_rect_list.append([x, y, w, h])
            
            # Debugging output
            print(f"Bounding Box: x={x}, y={y}, w={w}, h={h}, area={area}")

            # Check if area is within the valid range
            if area < min_area or area > max_area:
                print(f"Template Discarded - Area {area} is outside permitted range!")

        bounding_rect_list.sort()

        # Filter the bounding rectangles based on area
        res_list = [rect for rect in bounding_rect_list if rect[2] * rect[3] > min_area]

        # Eliminate rectangles that are inside bigger ones
        i = 0
        while i < len(res_list):
            [x, y, w, h] = res_list[i]
            j = i + 1
            while j < len(res_list):
                [x0, y0, w0, h0] = res_list[j]

                if (x + w) >= x0 + w0:
                    res_list.pop(j)
                else:
                    break
            i += 1

        # Further elimination based on position and height
        i = 0
        while i < len(res_list):
            [x, y, w, h] = res_list[i]

            if (h_img - (y + h)) > 40 or h < 17:
                res_list.pop(i)
            else:
                i += 1
        
        # Draw remaining rectangles
        for rect in res_list:
            [x, y, w, h] = rect
            cv2.rectangle(copy, (x, y), (x + w, y + h), (0, 0, 255), 1)        

        # Check if 9 rectangles (characters) are detected
        if len(res_list) == 9:
            test_passed = True
            res_img_list.append(copy)
            count += 1
            print('Test Successful: 9 letters found!')
        else:
            print('Unsuccessful!')

        # Stop if 3 successful cases are found
        if count == 3:
            break

    # Select the best image from successful cases
    if count == 0:
        best_img = crop_bgr
    else:
        best_img = res_img_list[min(count, 3) - 1]  # Use count to select the best image

    # Display final result
    if test_passed:
        print('Test Passed! 9 characters were detected in the serial number panel.')
        plt.imshow(best_img)
        plt.show()
    else:
        print('Test Failed! 9 characters were NOT detected in the serial number panel.')
    
    # Store the result globally
    global number_panel_result
    number_panel_result = [best_img, test_passed]
    
    # Update progress in the progress bar
    global myProgress
    progress['value'] = 90
    ProgressWin.update_idletasks()


# Results

In [ ]:
NUM_OF_FEATURES = 10  # Define the number of features globally
def testResult():
    plt.rcParams["figure.figsize"] = (3, 3)
    
    print('\n\nRESULT ANALYSIS\n')
    
    # Ensure the length of min_ssim_score_list matches the number of features
    min_ssim_score_list = [0.45, 0.4, 0.45, 0.45, 0.45, 0.4, 0.5]

    global result_list
    result_list = []
    successful_features_count = 0

    print("Length of avg_ssim_list:", len(avg_ssim_list))  # Debugging print
    print("Length of best_extracted_img_list:", len(best_extracted_img_list))  # Debugging print
    print("NUM_OF_FEATURES:", NUM_OF_FEATURES)

    # Ensure the loop does not exceed the number of features
    for i in range(min(len(avg_ssim_list), len(min_ssim_score_list))):  
        avg_score = avg_ssim_list[i]
        img, max_score = best_extracted_img_list[i]

        # No fallback needed, since loop is limited by list length
        min_ssim_score = min_ssim_score_list[i]

        status = avg_score >= min_ssim_score or max_score >= 0.79
        successful_features_count += status
        
        if status:
            print('Feature ' + str(i + 1) + ': successful')
        else:
            print('Feature ' + str(i + 1) + ': unsuccessful')
        
        img = img if img is not None else cv2.imread('Image_not_found.jpg')
        result_list.append([img, avg_score, max_score, status])

     # Feature 8: Left Bleed lines

    img, line_count = left_BL_result[:]

    # The feature passes the test if number of bleed lines is close to 7 (6.7 - 7.6)
    if line_count >= 6.7 and line_count <= 7.6:
        status = True
        successful_features_count += 1
        print('Feature 8: Successful- 7 bleed lines found in left part of currency note')
    else:
        status = False
        print('Feature 8: Unsuccessful!')
    
    if img is None:
        img = cv2.imread('Image_not_found.jpg')
    result_list.append([img, line_count, status])

    
    # Feature 9: Right Bleed lines

    img, line_count = right_BL_result[:]

    # The feature passes the test if number of bleed lines is close to 7 (6.7 - 7.6)
    if line_count >= 6.7 and line_count <= 7.6:
        status = True
        successful_features_count += 1
        print('Feature 9: Successful- 7 bleed lines found in right part of currency note')
    else:
         status = False
         print('Feature 9: Unsuccessful!')
    
    if img is None:
        img = cv2.imread('Image_not_found.jpg')
    result_list.append([img, line_count, status])

    
    # Feature 10: Currency Number Panel

    img, status = number_panel_result[:]

    # The feature passes the test if 9 characters are detected in the number panel
    if status:
        successful_features_count += 1
        print('Feature 10: Successful- 9 characters found in number panel of currency note')
    else:
        print('Feature 10: Unsuccessful!')
    
    if img is None:
        img = cv2.imread('Image_not_found.jpg')
    result_list.append([img, status])
 # printing FINAL RESULT

    print('\nResult Summary:')
    print(str(successful_features_count) + ' out of 10 features are VERIFIED!')
    
    # Updating progress bar
    global myProgress
    progress['value']=97
    ProgressWin.update_idletasks()

    print("Total successful features:", successful_features_count)

    if successful_features_count != NUM_OF_FEATURES:
        if os.name == 'nt':
            os.system('echo \a')  # Trigger a beep sound
    is_real = successful_features_count == NUM_OF_FEATURES 
    def display_result(is_real):
    # Create the main window
        root = tk.Tk()
        root.title("Currency Verification Result")
        root.geometry("400x200")  # Set the size of the window

    # Define the font style for the result
        result_font = font.Font(family="Helvetica", size=40, weight="bold")

    # Create a label to display the result
        result_label = tk.Label(root, text="REAL" if is_real else "FAKE", font=result_font)
    
    # Set the color of the text
        result_label.config(fg="green" if is_real else "red")
    
    # Add the label to the window
        result_label.pack(expand=True)

    # Run the Tkinter main loop
        root.mainloop()

# Example usage
 # Set this variable to True if the note is real, False if fake
    display_result(is_real)


# Driver cell

In [ ]:
# Driver cell- contains GUI elements

# Call all testing functions
def Testing():  
    button.config(state = DISABLED)
    result_list.clear()
    preprocessing()
    testFeature_1_2_7()
    testFeature_8()
    testFeature_9()
    testFeature_10()
    testResult()
    progress['value']=100
    ProgressWin.update_idletasks()
    time.sleep(0.8)
    ProgressWin.destroy()

def exitGUI():
    ProgressWin.destroy()


# creating tkinter window 
ProgressWin = Tk() 
ProgressWin.title("Processing Image")
ProgressWin.title('Fake Currency Detection - Processing')

# Defining attributes of root window
ProgressWin.resizable(False, False)  # This code helps to disable windows from resizing

window_height = 200
window_width = 500

screen_width = ProgressWin.winfo_screenwidth()
screen_height = ProgressWin.winfo_screenheight()

x_cordinate = int((screen_width/2) - (window_width/2))
y_cordinate = int((screen_height/2) - (window_height/2))

ProgressWin.geometry("{}x{}+{}+{}".format(window_width, window_height, x_cordinate, y_cordinate))

# Creating a main frame inside the root window
main_frame=Frame(ProgressWin,relief=GROOVE)
main_frame.place(x=10,y=10) # Placing the frame at (10, 10)

# Creating sub- frames
frame1 = Frame(main_frame, padx=3, pady=3)
frame2 = Frame(main_frame, bg='dark blue', pady=5, padx = 5)
frame3 = Frame(main_frame, pady=5, padx = 5)

frame1.grid(row = 1, column = 1, padx = 5, pady = 5)
frame2.grid(row = 2, column = 1, padx = 5, pady = 5)
frame3.grid(row = 3, column = 1, padx = 5, pady = 30)


# Title label in sub_frame1
label = Label(master=frame1, text="Processing! Please wait...", fg = 'green', font = "Verdana 13 bold")
label.pack() # Put the label into the window

# Progress bar widget 
progress = Progressbar(frame2, orient = HORIZONTAL, length = 450, mode = 'determinate') 
progress.pack()

# Button widget
button = Button(frame3, text = 'Click to continue', command = Testing, font = "Verdana 12 bold", pady = 5)
button.pack()

# Run the GUI  
ProgressWin.mainloop()

In [ ]:
# Store the list containing the final result of each feature

%store result_list